# Иерархия для Постнауки. Version 0.

In [1]:
import artm
import os
import glob
import sys

In [2]:
sys.path.append("D:/PTM/code/BigAltHier")

In [86]:
import hierarchy_utils
reload(hierarchy_utils)

<module 'hierarchy_functions' from 'D:/PTM/code/BigAltHier\hierarchy_functions.py'>

In [57]:
data_path_UCI = 'ngramm/pymorphy/UCI_trimmed/'
data_path_batches = 'ngramm/pymorphy/batches/'
if len(glob.glob(data_path_batches + "/*.batch")) < 1:
    batch_vectorizer = artm.BatchVectorizer(collection_name='postnauka', data_format='bow_uci', \
                data_path = data_path_UCI, batch_size = 200, target_folder=data_path_batches)
else:
    batch_vectorizer = artm.BatchVectorizer(data_path=data_path_batches, data_format='batches')

In [6]:
def get_topic_names(topicNum):
    """
    generates topic names
    """
    n_bckgrnd = 0
    n_subj = topicNum-n_bckgrnd
    topics_bckgrnd = [u'background_'+str(i).zfill(3) for i in range(n_bckgrnd)]
    topics_subj = [u'subj_'+str(i).zfill(3) for i in range(n_subj)]
    topic_names = topics_bckgrnd + topics_subj
    return topic_names

In [7]:
class_ids={'ngramm':1.0, 'projects':1.0, 'category':1, 'author': 5.0, 'post_tag':5.0}

In [10]:
regularizers_list = []

regularizers_list.append(artm.SmoothSparsePhiRegularizer(name='SPPhiCatReg',class_ids=['category'],tau=0.05))
regularizers_list.append(artm.SmoothSparsePhiRegularizer(name='SPPhiAuthorReg',class_ids=['author'],tau=0.05))
regularizers_list.append(artm.SmoothSparsePhiRegularizer(name='SPPhiTagReg',class_ids=['post_tag'],tau=0.05))
regularizers_list.append(artm.DecorrelatorPhiRegularizer(name='DecorrPhiReg',class_ids=['ngramm'],tau=50000))

regularizers_list.append(artm.SmoothSparsePhiRegularizer(name='SPPhiNgrammRegSubj',
                                                         class_ids=['ngramm'],
                                                         #topic_names=topics_subj,
                                                         tau=0.01))
regularizers_list.append(artm.SmoothSparseThetaRegularizer(name='SmoothThetaRegSubj',
                                                           #topic_names=topics_subj,
                                                           tau=0.01))

In [13]:
# quality measures of models
scores_list = []
scores_list.append(artm.SparsityPhiScore(name='SparsityScoreNgrammSubj',
                                         class_id='ngramm',
                                         #topic_names=topics_subj
                                        ))
scores_list.append(artm.SparsityThetaScore(name='SparsityScoreThetaSubj',
                                           #topic_names=topics_subj
                                          ))

scores_list.append(artm.SparsityPhiScore(name='SparsityScoreCat', class_id='category'))
scores_list.append(artm.SparsityPhiScore(name='SparsityScoreAuthor', class_id='author'))
scores_list.append(artm.SparsityPhiScore(name='SparsityScoreTag', class_id='post_tag'))
scores_list.append(artm.TopTokensScore(name='TopTokensScoreNgramm', class_id='ngramm', num_tokens=10))
scores_list.append(artm.TopTokensScore(name='TopTokensTag', class_id='post_tag', num_tokens=10))
scores_list.append(artm.TopicKernelScore(name='TopicKernelNgramm', class_id='ngramm', probability_mass_threshold=0.25))
scores_list.append(artm.PerplexityScore(name='PerplexityScore', class_ids=['ngramm']))

In [58]:
dict_path = 'ngramm/pymorphy/UCI_trimmed/dictionary.txt'
if not os.path.isfile(dict_path):
    dictionary = artm.Dictionary('dictionary')
    dictionary.gather(batch_vectorizer.data_path)
    dictionary.save_text(dict_path)
else:
    dictionary = artm.Dictionary('dictionary')
    dictionary.load_text(dict_path)

### Построение первого уровня (обыкновеная тематическая модель)

In [119]:
topicNum0 = 10
level0 = artm.ARTM(num_topics=topicNum0,
                  topic_names=get_topic_names(topicNum0),
                  class_ids=class_ids,
                  regularizers=regularizers_list,
                  scores=scores_list,
                  seed=37)
level0.initialize(dictionary=dictionary)

In [120]:
level0.fit_offline(batch_vectorizer, num_collection_passes=30)

In [121]:
tokens = level0.score_tracker["TopTokensTag"].last_tokens
for topic_name in level0.topic_names:
    print topic_name + ': ',
    for word in tokens[topic_name]:    
        print word,
    print

subj_000:  биология антропология эволюция человек история палеонтология антропогенез география геология археология
subj_001:  физика астрономия астрофизика Вселенная звезды физика_элементарных_частиц галактика квантовая_физика космос черные_дыры
subj_002:  культура социология общество массовая_культура город культурология философия кино искусство гендер
subj_003:  биология медицина генетика ДНК ген клетка геном технологии микробиология химия
subj_004:  история политика история_России право Россия Средневековье государство СССР политология Германия
subj_005:  экономика история Россия США религия Европа государство христианство общество Китай
subj_006:  философия социология образование наука Россия университет история_науки общество история_философии Кант_Иммануил
subj_007:  психология мозг технологии информационные_технологии нейробиология биология нейрон когнитивная_психология нейрофизиология мышление
subj_008:  математика физика Земля химия материаловедение Солнечная_система планета а

### Построение второго уровня (класс ARTM_Level)
__Очень важно:__ при создании каждого уровня указывать новый seed (иначе дочерние темы начинают дублировать родительские)

In [132]:
topicNum1 = 30
level1 = hierarchy_functions.ARTM_Level(level0, phi_batch_weight=10.0**3, 
                                        num_topics=topicNum1, 
                                        topic_names=get_topic_names(topicNum1), 
                                         class_ids=class_ids,
                                          regularizers=regularizers_list,
                                           scores=scores_list,
                                            num_document_passes=1, 
                                             cache_theta=True, seed=124)

In [133]:
level1.initialize(dictionary=dictionary)
level1.fit_offline(batch_vectorizer, num_collection_passes=30)

In [124]:
tokens = level1.score_tracker["TopTokensTag"].last_tokens
for topic_name in level1.topic_names:
    print topic_name + ': ',
    for word in tokens[topic_name]:    
        print word,
    print

subj_000:  астрофизика астрономия черные_дыры звезды галактика Вселенная физика гравитация физика_элементарных_частиц нейтронные_звезды
subj_001:  мозг нейробиология нейрон нейрофизиология память искусственный_интеллект психология физиология эмоции робототехника
subj_002:  общество город Россия урбанистика искусство городское_пространство архитектура Москва культура социология_пространства
subj_003:  астрономия космос Солнечная_система планета Солнце звезды Земля Вселенная галактика Марс
subj_004:  образование наука университет история_науки школа академическая_среда социология_науки философия_науки Россия ЕГЭ
subj_005:  культура массовая_культура культурология кино гендер кинематограф театр СССР идентичность театроведение
subj_006:  физика Земля экзопланета материаловедение Эйнштейн_Альберт химия метеорит квантовая_механика история_науки марсоход
subj_007:  физика Вселенная астрофизика космология астрономия космическая_инфляция реликтовое_излучение космос Большой_взрыв сверхпроводимос

In [137]:
psi1 = level1.get_psi()

In [138]:
psi1   # p(subtopic|topic)

,subj_000,subj_001,subj_002,subj_003,subj_004,subj_005,subj_006,subj_007,subj_008,subj_009
subj_000,0.008903,0.229029,0.004640,0.010050,0.005041,0.007278,0.003019,0.006533,0.027819,0.006353
subj_001,0.007840,0.010365,0.006434,0.014970,0.006199,0.006266,0.005720,0.263814,0.008386,0.006189
subj_002,0.015374,0.004723,0.172318,0.004196,0.037961,0.059981,0.013505,0.009068,0.012131,0.008350
subj_003,0.012169,0.071048,0.007238,0.005809,0.009032,0.009916,0.004995,0.010210,0.187229,0.007910
subj_004,0.007666,0.006506,0.007339,0.005385,0.009342,0.011474,0.232684,0.011365,0.007532,0.025904
subj_005,0.013643,0.004218,0.292286,0.003588,0.015898,0.010446,0.011084,0.006665,0.009387,0.021801
subj_006,0.016108,0.050915,0.007140,0.015384,0.008397,0.008565,0.007183,0.013139,0.160155,0.009041
subj_007,0.010178,0.187102,0.006856,0.011252,0.006208,0.007209,0.004461,0.012316,0.027354,0.006803
subj_008,0.013772,0.005431,0.016799,0.004619,0.241001,0.046744,0.011429,0.006937,0.009950,0.009791
subj_009,0.010402,0.008408,0.010208,0.008454,0.010738,0.031749,0.009320,0.135846,0.047626,0.007719


#### Выведем подтемы тем первого уровня (с порогом p(subtopic|topic) > 0.05)

In [127]:
tokens0 = level0.score_tracker["TopTokensTag"].last_tokens
tokens1 = level1.score_tracker["TopTokensTag"].last_tokens
for t, topic_name in enumerate(level0.topic_names):
    print topic_name + ': ',
    for word in tokens0[topic_name]:    
        print word,
    print
    for s, topic_name1 in enumerate(level1.topic_names):
        if psi.values[s, t] > 0.05:
            print "\t", topic_name1 + ': ',
            for word in tokens1[topic_name1]:    
                print word,
            print  

subj_000:  биология антропология эволюция человек история палеонтология антропогенез география геология археология
	subj_011:  антропология человек антропогенез археология происхождение_человека Африка майя расоведение этнография неандерталец
	subj_016:  история Древняя_Греция античность христианство культура востоковедение нация миграция этнос мифология
	subj_022:  биология микробиология экология география микробы геология палеонтология биотехнологии бактерии климат
	subj_027:  биология эволюция зоология этология поведение_животных Дарвин_Чарльз эволюционная_биология теория_эволюции зоопсихология палеонтология
subj_001:  физика астрономия астрофизика Вселенная звезды физика_элементарных_частиц галактика квантовая_физика космос черные_дыры
	subj_000:  астрофизика астрономия черные_дыры звезды галактика Вселенная физика гравитация физика_элементарных_частиц нейтронные_звезды
	subj_003:  астрономия космос Солнечная_система планета Солнце звезды Земля Вселенная галактика Марс
	subj_006:  

### Построение третьего уровня

In [134]:
topicNum2 = 70
level2 = hierarchy_functions.ARTM_Level(level1, phi_batch_weight=10.0**3, 
                                        num_topics=topicNum2, 
                                        topic_names=get_topic_names(topicNum2), 
                                         class_ids=class_ids,
                                          regularizers=regularizers_list,
                                           scores=scores_list,
                                            num_document_passes=1, 
                                             cache_theta=True, seed=741)

In [135]:
level2.initialize(dictionary=dictionary)
level2.fit_offline(batch_vectorizer, num_collection_passes=30)

In [139]:
psi2 = level2.get_psi()

In [141]:
tokens2 = level2.score_tracker["TopTokensTag"].last_tokens
for topic_name in level2.topic_names:
    print topic_name + ': ',
    for word in tokens2[topic_name]:    
        print word,
    print

subj_000:  город урбанистика городское_пространство общество искусство архитектура Москва социология_пространства авангард искусствоведение
subj_001:  клетка биология онкология стволовые_клетки клеточная_биология рак индуцированные_плюрипотентные_стволовые_клетки раковые_клетки регенеративная_медицина квантовая_химия
subj_002:  эволюция биология этология зоология поведение_животных Дарвин_Чарльз эволюционная_биология зоопсихология теория_эволюции приматы
subj_003:  США экономика социальная_структура финансовый_рынок безработица Нобелевская_премия Россия средний_класс рынок_труда экология
subj_004:  Средневековье христианство юриспруденция римское_право католицизм Рим Европа право медиевистика история_права
subj_005:  филология фольклористика фольклор поэзия народная_культура Скандинавия русская_литература коммуникация письменность литература
subj_006:  Солнечная_система планета Солнце космос астрономия Марс экзопланета планеты Земля Юпитер
subj_007:  материаловедение история_науки мета

### Генерация json

In [149]:
import json

In [170]:
psis = [psi1, psi2]
tokens = [tokens0, tokens1, tokens2]
levels = [level0, level1, level2]
max_level = 2
def gen_json_level(level, topic, supertopic=None, threshold=0.05, num_tokens=5):
    if level == max_level:
        return {"size": int(psis[-1].values[topic, supertopic]*10000), 
                "name": ", ".join(tokens[-1][levels[-1].topic_names[topic]][:num_tokens]).encode("utf-8")}
    json = {"name": ", ".join(tokens[level][levels[level].topic_names[topic]][:num_tokens]).encode("utf-8"),
           "children": [gen_json_level(level+1, subtopic, topic) 
                        for subtopic in range(levels[level+1].num_topics)
                        if psis[level].values[subtopic, topic] > threshold]}
    if level != 0:
        json["size"] = int(psis[level-1].values[topic, supertopic]*10000)
    return json

In [171]:
h_json = {"children": [gen_json_level(0, subtopic) for subtopic in range(level0.num_topics)]}

In [178]:
with open("./hier.json", "w") as fout:
    json.dump(h_json, fout, ensure_ascii=False)